# Python Intro - Final Exercise


In [1]:
import pyodbc
import pandas as pd
import numpy as np

In [2]:
### In windows, Using a ODBC DNS (predefined connection name)
import pyodbc
conn = pyodbc.connect ("DSN=College;"
                      "Server=DESKTOP-AAGNMGA\SQLEXPRESS;"
                      "Database=College;"
                      "Trusted_Connection=yes;")

In [3]:
### Get the students table
students = pd.read_sql("SELECT * FROM Students;",conn)
students.head()

,StudentId,FirstName,LastName,Gender
0,162.0,Elmira,Holston,F
1,154.0,Sierra,Van Vorst,F
2,28.0,Mycah,Bingham,F
3,54.0,Alia,Jones,F
4,31.0,Chanda,Fillmore,F


In [4]:
departments = pd.read_sql("SELECT * FROM Departments;",conn)
departments.head()

,DepartmentId,DepartmentName
0,1.0,English
1,2.0,Science
2,3.0,Arts
3,4.0,Sport


In [5]:
courses = pd.read_sql("SELECT * FROM Courses;",conn)
courses.head()

,CourseId,CourseName,DepartmentId,TeacherId
0,1.0,English Begginers,1.0,15.0
1,2.0,Advanced English,1.0,15.0
2,3.0,Proffesional English,1.0,15.0
3,4.0,Mathematics 101,2.0,2.0
4,5.0,Matematics Advanced,2.0,13.0


In [6]:
classrooms = pd.read_sql("SELECT * FROM Classrooms;",conn)
classrooms.head()

,CourseId,StudentId,degree
0,1.0,202.0,92.031070
1,1.0,12.0,81.175916
2,1.0,232.0,82.396587
3,1.0,123.0,62.420449
4,1.0,274.0,72.569900


In [7]:
teachers = pd.read_sql("SELECT * FROM Teachers;",conn)
teachers.head()

,TeacherId,FirstName,LastName,Gender
0,1.0,Kaylea,Monzingo,F
1,2.0,Amanda,Dworkin,F
2,3.0,Jordyn,Baron,F
3,4.0,Katherine,Dibari,F
4,5.0,Heather,Metcalf,F


# Questions
 
 
## Q1. Count the number of students on each department

In [8]:
Q1 = classrooms.merge(courses, on = 'CourseId').merge(departments, on='DepartmentId')
Q1.head()

,CourseId,StudentId,degree,CourseName,DepartmentId,TeacherId,DepartmentName
0,1.0,202.0,92.031070,English Begginers,1.0,15.0,English
1,1.0,12.0,81.175916,English Begginers,1.0,15.0,English
2,1.0,232.0,82.396587,English Begginers,1.0,15.0,English
3,1.0,123.0,62.420449,English Begginers,1.0,15.0,English
4,1.0,274.0,72.569900,English Begginers,1.0,15.0,English


In [9]:
Q1.groupby('DepartmentName').agg({'StudentId':pd.Series.nunique})


,StudentId
DepartmentName,
Arts,137.0
English,84.0
Science,200.0
Sport,81.0


## Q2. How many students have each course of the English department and the total number of students in the department?

In [10]:
Q1[(Q1.DepartmentId.isin([1]))].\
    groupby('CourseName').\
    agg({'StudentId':pd.Series.nunique})

,StudentId
CourseName,
Advanced English,24.0
English Begginers,32.0
Proffesional English,38.0


In [11]:
Q1[(Q1.DepartmentId.isin([1]))].\
    groupby('DepartmentName').\
    agg({'StudentId':pd.Series.nunique})

,StudentId
DepartmentName,
English,84.0


## Q3. How many small (<22 students) and large (22+ students) classrooms are needed for the Science department?

In [12]:
Q3 = Q1[(Q1.CourseId.between(4,19))].\
    groupby(['DepartmentName','CourseName']).\
    agg({'StudentId':pd.Series.nunique})
Q3["ClassSize"]='Small Class'
Q3.loc[Q3["StudentId"]>22,"ClassSize"]='Big Class'
Q3


StudentId    ClassSize
DepartmentName CourseName                                 
Science        Algebra                   12.0  Small Class
               Biology                   36.0    Big Class
               Chemistry                 23.0    Big Class
               Computer Science          38.0    Big Class
               Data Science              30.0    Big Class
               Genetics                  37.0    Big Class
               Geometry                  13.0  Small Class
               Java                      16.0  Small Class
               Matematics Advanced       10.0  Small Class
               Mathematics 101           18.0  Small Class
               Neurobiology              24.0    Big Class
               Physics                   12.0  Small Class
               Python                    23.0    Big Class
               R                         33.0    Big Class
               SQL                       24.0    Big Class
               Statistics                24.0    Big Class

In [13]:
Q3.groupby('ClassSize').agg({'ClassSize':'count'})

,ClassSize
ClassSize,
Big Class,10
Small Class,6


## Q4. A feminist student claims that there are more male than female in the College. Justify if the argument is correct

In [14]:
students.groupby('Gender').agg({'Gender':'count'})

,Gender
Gender,
F,165
M,115


## Q5. For which courses the percentage of male/female students is over 70%?

In [15]:
Q5 = classrooms.merge(students, on = 'StudentId', how='left').merge(courses, on='CourseId', how='left')
Q5.head()

,CourseId,StudentId,degree,FirstName,LastName,Gender,CourseName,DepartmentId,TeacherId
0,1.0,202.0,92.031070,Autumn,Major,F,English Begginers,1.0,15.0
1,1.0,12.0,81.175916,Sophiya,Aponte,F,English Begginers,1.0,15.0
2,1.0,232.0,82.396587,Victoria,Kishinevsky,F,English Begginers,1.0,15.0
3,1.0,123.0,62.420449,Kelsi,Taber,F,English Begginers,1.0,15.0
4,1.0,274.0,72.569900,Samantha,Anderson,F,English Begginers,1.0,15.0


In [16]:
over_70per = Q5.groupby(['CourseName','Gender']).\
            agg({'Gender':'count'}) / Q5.groupby(['CourseName']).agg({'Gender':'count'})

In [17]:
over_70per[(over_70per.Gender.between(0.7001,1))].\
            groupby(['CourseName']).\
            agg({'Gender':[min]})


,Gender
,min
CourseName,
Sculpture,0.708333
Tenis,0.722222


## Q6. For each department, how many students passed with a grades over 80?

In [18]:
Above80=Q1[Q1.degree.between(80,100)].\
        groupby(['DepartmentName']).\
        agg({'StudentId':pd.Series.nunique}) /Q1.groupby(['DepartmentName']).agg({'StudentId':pd.Series.nunique})*100
Above80

,StudentId
DepartmentName,
Arts,44.525547
English,47.619048
Science,61.000000
Sport,48.148148


## Q7. For each department, how many students passed with a grades under 60?

In [19]:
Under60=Q1[Q1.degree.between(0,60)].\
        groupby(['DepartmentName']).\
        agg({'StudentId':pd.Series.nunique}) /Q1.groupby(['DepartmentName']).agg({'StudentId':pd.Series.nunique})*100
Under60

,StudentId
DepartmentName,
Arts,2.189781
English,2.380952
Science,3.500000
Sport,1.234568


## Q8. Rate the teachers by their average student's grades (in descending order).

In [20]:
Q8= classrooms.merge(courses, on = 'CourseId').merge(teachers, on='TeacherId')
Q8['TeacherName'] = Q8['FirstName'] + ' ' + Q8['LastName']


In [21]:
Q8.groupby(['TeacherName',]).\
agg({'degree':'mean'}).sort_values(by='degree', ascending=False)


,degree
TeacherName,
Darick Hess,81.432703
Jordyn Baron,80.729494
Heather Metcalf,80.440242
Zachary Healy,79.365867
Kaylea Monzingo,79.365537
Daniel Bland,78.908281
Keegan Price,78.814631
Conor Mckittrick,78.699492
Maxwell Romkee,78.673509


## Q9. Create a dataframe showing the courses, departments they are associated with, the teacher in each course, and the number of students enrolled in the course (for each course, department and teacher show the names).

In [22]:
Q9 = courses.merge(departments, on = 'DepartmentId', how='left').\
             merge(classrooms, on='CourseId', how='left').\
             merge(teachers, on='TeacherId', how='left')

In [23]:
Q9.groupby(['CourseId','CourseName','DepartmentName','FirstName','LastName']).\
   agg({'StudentId':pd.Series.nunique})

,,,,,StudentId
CourseId,CourseName,DepartmentName,FirstName,LastName,
1.0,English Begginers,English,Conor,Mckittrick,32.0
2.0,Advanced English,English,Conor,Mckittrick,24.0
3.0,Proffesional English,English,Conor,Mckittrick,38.0
4.0,Mathematics 101,Science,Amanda,Dworkin,18.0
5.0,Matematics Advanced,Science,Alexander,Watts,10.0
6.0,Algebra,Science,Alexander,Watts,12.0
7.0,Geometry,Science,Alexander,Watts,13.0
8.0,Chemistry,Science,Jeffrey,Batzel,23.0
9.0,Physics,Science,Alexander,Watts,12.0


## Q10. Create a dataframe showing the students, the number of courses they take, the average of the grades per class, and their overall average (for each student show the student name).

In [51]:
Q10 = students.merge(classrooms, on = 'StudentId', how='left').\
               merge(courses, on='CourseId', how='left')
Q10.head()

,StudentId,FirstName,LastName,Gender,CourseId,degree,CourseName,DepartmentId,TeacherId
0,162.0,Elmira,Holston,F,8.0,65.987050,Chemistry,2.0,14.0
1,162.0,Elmira,Holston,F,20.0,79.476271,History of Arts,3.0,8.0
2,162.0,Elmira,Holston,F,31.0,74.546820,Gymnastics,4.0,23.0
3,154.0,Sierra,Van Vorst,F,6.0,84.137367,Algebra,2.0,13.0
4,154.0,Sierra,Van Vorst,F,18.0,81.048019,R,2.0,17.0


In [52]:
Overall_Average = Q10.groupby(['StudentId','FirstName','LastName']).\
                      agg({'CourseId':pd.Series.nunique,'degree':'mean'})
Overall_Average.reset_index(inplace=True)
Overall_Average.head()

,StudentId,FirstName,LastName,CourseId,degree
0,1.0,Alexandria,Heller,4.0,81.337515
1,2.0,Cassie,Liska,5.0,79.137364
2,3.0,Hayley,Frank,3.0,78.793700
3,4.0,Maxwell,Kotch,3.0,71.481003
4,5.0,Dylan,Boehm,3.0,76.718650


In [53]:
Q10['Average']='English_Degree'
Q10.loc[Q10['DepartmentId']==2,'Average']='Science_Degree'
Q10.loc[Q10['DepartmentId']==3,'Average']='Art_Degree'
Q10.loc[Q10['DepartmentId']==4,'Average']='Sport_Degree'

DepartmentAvg = Q10.groupby(['StudentId','FirstName','LastName','DepartmentId','Average']).agg({'CourseId':pd.Series.nunique,'degree':'mean'})

DepartmentAvg.head()

#ליצור 4 DF. אחד לכל מחלקה ואז לעשות להם LEFT JIONT

CourseId  \
StudentId FirstName   LastName DepartmentId Average                    
1.0        Alexandria Heller   2.0          Science_Degree       3.0   
                               3.0          Art_Degree           1.0   
2.0        Cassie     Liska    2.0          Science_Degree       3.0   
                               3.0          Art_Degree           2.0   
3.0        Hayley     Frank    1.0          English_Degree       1.0   

                                                               degree  
StudentId FirstName   LastName DepartmentId Average                    
1.0        Alexandria Heller   2.0          Science_Degree  79.557508  
                               3.0          Art_Degree      86.677535  
2.0        Cassie     Liska    2.0          Science_Degree  78.867115  
                               3.0          Art_Degree      79.542738  
3.0        Hayley     Frank    1.0          English_Degree  83.972245

In [71]:
Overall_Average.reindex(columns = np.append( Overall_Average.columns.values, ['English_Degree','Science_Degree','Art_Dgree','Sport_Degree']))
#Overall_Average['English_Degree']=np.where(DepartmentAvg['DepartmentId']==1.0,np.mean,np.NaN)
#Overall_Average.loc[Q10['DepartmentId']==1,'English_Degree'] = Q10.groupby(['StudentId']).agg({'degree':'mean'})
#Overall_Average.loc[Q10['DepartmentId']==2,'Art_Degree'] = Q10.groupby(['StudentId','FirstName','LastName','DepartmentId','Average']).agg({'degree':'mean'})
#Overall_Average.loc[Q10['DepartmentId']==3,'Science_Degree'] = Q10.groupby(['StudentId','FirstName','LastName','DepartmentId','Average']).agg({'degree':'mean'})
#Overall_Average.loc[Q10['DepartmentId']==4,'Sport_Degree'] = Q10.groupby(['StudentId','FirstName','LastName','DepartmentId','Average']).agg({'degree':'mean'})
Overall_Average

,StudentId,FirstName,LastName,CourseId,degree,English_Degree,Art_Degree,Science_Degree,Sport_Degree
0,1.0,Alexandria,Heller,4.0,81.337515,NaN,NaN,NaN,NaN
1,2.0,Cassie,Liska,5.0,79.137364,NaN,NaN,NaN,NaN
2,3.0,Hayley,Frank,3.0,78.793700,NaN,NaN,NaN,NaN
3,4.0,Maxwell,Kotch,3.0,71.481003,NaN,NaN,NaN,NaN
4,5.0,Dylan,Boehm,3.0,76.718650,NaN,NaN,NaN,NaN
5,6.0,Chelsea,Vuolo,2.0,71.578832,NaN,NaN,NaN,NaN
6,7.0,Katrina,Evers,4.0,73.325298,NaN,NaN,NaN,NaN
7,8.0,Sean,Hasenack,3.0,88.198366,NaN,NaN,NaN,NaN
8,9.0,Madalyn,Miller,2.0,86.802830,NaN,NaN,NaN,NaN
9,10.0,Katrina,Miller,4.0,80.622716,NaN,NaN,NaN,NaN
